In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,MinMaxScaler
import pickle

In [37]:
# Load Dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
#Preprocess the data
#Drop irrelevant Columns
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [39]:
#Encode Categorical Variables
label_encode_gender =LabelEncoder()
data['Gender'] = label_encode_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [40]:
data.describe()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [41]:
## One hot encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geo=OneHotEncoder()
geo_encoder =onehot_encode_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [42]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [43]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [44]:
onehot_encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [45]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [46]:
#Combine one hot encoder columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [47]:
## Save the encoders and sscaler:
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encode_geo,file)
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file)

In [48]:
#Divide the Data set to X and y
X= data.drop('Exited',axis=1)
y=data['Exited']

In [49]:
#Split the data into test and training sets 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scale these Features 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
# Save scaler in pkl file
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [52]:
X_train.shape

(8000, 12)

# ANN Implementation

import tensorflow as tf

In [58]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [53]:
## Build our ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')   
]

)

In [54]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [56]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [57]:
#compile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [59]:
#Setup the Tensorboard
log_dir="logs/fit"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [60]:
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [61]:
#Set up Early stopping

early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [64]:
### Train the model

history = model.fit(
    X_train,y_train,epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 1s 3ms/step - loss: 0.3958 - accuracy: 0.8371
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3552 - accuracy: 0.8558
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3486 - accuracy: 0.8576
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3427 - accuracy: 0.8609
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3406 - accuracy: 0.8622
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3385 - accuracy: 0.8597
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3364 - accuracy: 0.8615
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3317 - accuracy: 0.8651
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3325 - accuracy: 0.8639
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3301 - accu

In [65]:
model.save('model.h5')

c:\Users\tgunda976\Desktop\Daily_Tasks\Project\Deep Learning\Krishnaik\practice\tf_mydl\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
# Load the Model
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit20241130-083141

Reusing TensorBoard on port 6009 (pid 24216), started 0:00:26 ago. (Use '!kill 24216' to kill it.)

: 